In [0]:
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd
from scipy import stats
import os,sys

In [0]:
#To get access to the files on google drive an authorization code it nessacery which will be given by the linked below
drive.mount('/gdrive')

In [0]:
input_data_path="/gdrive/My Drive/Colab Notebooks/Input_files/produced/"
output_data_path="/gdrive/My Drive/Colab Notebooks/Output_files"

In [4]:
df=pd.read_csv(os.path.join(input_data_path,'data_InstaCart.csv'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33894106 entries, 0 to 33894105
Data columns (total 12 columns):
order_id                  int64
user_id                   int64
eval_set                  object
order_number              int64
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
product_id                float64
add_to_cart_order         float64
reordered                 float64
aisle_id                  float64
department_id             float64
dtypes: float64(6), int64(5), object(1)
memory usage: 3.0+ GB


In [0]:
df=df.loc[df.eval_set!='test',['user_id','product_id']]

In [6]:
df.head()

,user_id,product_id
0,1,196.0
1,1,12427.0
2,1,14084.0
3,1,26088.0
4,1,26405.0


In [7]:
data=pd.DataFrame({'user_portion':df.groupby(['user_id','product_id']).size()}).reset_index()
data.head()

,user_id,product_id,user_portion
0,1,196.0,11
1,1,10258.0,10
2,1,10326.0,1
3,1,12427.0,10
4,1,13032.0,4


In [8]:
data=data.join(pd.DataFrame({'product_quantity':df.groupby('user_id').size()}).reset_index().set_index('user_id'),on=('user_id'))
data.head()

,user_id,product_id,user_portion,product_quantity
0,1,196.0,11,70
1,1,10258.0,10,70
2,1,10326.0,1,70
3,1,12427.0,10,70
4,1,13032.0,4,70


In [0]:
del df
data.loc[:,'rating']=data.user_portion/data.product_quantity

In [0]:
data=data.join(pd.DataFrame({'max_rating':data.groupby('user_id').rating.max()}).reset_index().set_index('user_id'),on=('user_id'))


In [11]:
data.head()

,user_id,product_id,user_portion,product_quantity,rating,max_rating
0,1,196.0,11,70,0.157143,0.157143
1,1,10258.0,10,70,0.142857,0.157143
2,1,10326.0,1,70,0.014286,0.157143
3,1,12427.0,10,70,0.142857,0.157143
4,1,13032.0,4,70,0.057143,0.157143


In [0]:
data.rating=(data.rating*100/data.max_rating).round()

In [13]:
data.head()

,user_id,product_id,user_portion,product_quantity,rating,max_rating
0,1,196.0,11,70,100.0,0.157143
1,1,10258.0,10,70,91.0,0.157143
2,1,10326.0,1,70,9.0,0.157143
3,1,12427.0,10,70,91.0,0.157143
4,1,13032.0,4,70,36.0,0.157143


In [0]:
data=data.loc[:,['user_id','product_id','rating']]

In [15]:
data.head()

,user_id,product_id,rating
0,1,196.0,100.0
1,1,10258.0,91.0
2,1,10326.0,9.0
3,1,12427.0,91.0
4,1,13032.0,36.0


In [0]:
data.to_csv(os.path.join(input_data_path,'data_InstaCart_rating.csv'), encoding='utf-8',index=False)